In [2]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from collections import deque

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

import tests.qubx.ta.utils_for_testing as test
from qubx.utils import fig, sbp

from qubx.data.readers import (
    CsvDataReader, QuotesDataProcessor, OhlcvDataProcessor, QuotesFromOHLCVDataProcessor, 
    OhlcvPandasDataProcessor, STOCK_DAILY_SESSION
)
from qubx.core.series import TimeSeries, OHLCV
from qubx.ta.indicators import (sma, ema, tema, dema, kama, highest, lowest)


                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



## <font color='gray'>Reload dev environment</font>

In [7]:
from qubx.core.series import TimeSeries; from qubx.ta.indicators import sma; del sys.modules['qubx.core.series']; del sys.modules['qubx.ta.indicators']
from qubx.core.series import TimeSeries, OHLCV
from qubx.core.utils import recognize_time
from qubx.ta.indicators import (sma, ema, tema, dema, kama, highest, lowest)

## <font color='red'><b>On formed bar experiments</b></font>

In [22]:
r2x = CsvDataReader('../tests/data/csv/BTCUSDT_ohlcv_M1_sec.csv.gz', QuotesFromOHLCVDataProcessor(trades=0))
ticks = r2x.read()
# - small storage
# r0 = CsvDataReader('../tests/data/csv/quotes.csv', QuotesDataProcessor())
# ticks = r0.read()
len(ticks)

260712

In [27]:
s0 = TimeSeries('T0', '30Sec', process_every_update=False)
control = TimeSeries('T0', '30Sec')
m0 = ema(s0, 5)
for q in ticks:
    s0.update(q.time, 0.5*(q.ask + q.bid))
    control.update(q.time, 0.5*(q.ask + q.bid))

m1 = ema(control, 5)
mx = test.scols(s0.pd(), control.pd(), m0.pd(), m1.pd(), names=[
    'series', 'control', 'streamed', 'finished'
]).dropna()
assert test.N(mx.streamed) == mx.finished
mx

,series,control,streamed,finished
2024-01-01 00:01:30,42350.4,42350.4,33860.740000,33860.740000
2024-01-01 00:02:00,42349.6,42349.6,36690.360000,36690.360000
2024-01-01 00:02:30,42360.2,42360.2,38580.306667,38580.306667
2024-01-01 00:03:00,42360.1,42360.1,39840.237778,39840.237778
2024-01-01 00:03:30,42405.8,42405.8,40695.425185,40695.425185
...,...,...,...,...
2024-02-15 06:15:00,51983.2,51983.2,51974.665555,51974.665555
2024-02-15 06:15:30,51995.9,51995.9,51981.743703,51981.743703
2024-02-15 06:16:00,52014.7,52014.7,51992.729135,51992.729135
2024-02-15 06:16:30,51966.0,51966.0,51983.819424,51983.819424
